In [88]:
import numpy as np
import transformers as tr
import torch

In [219]:
tokenizer = tr.AutoTokenizer.from_pretrained("ktrapeznikov/albert-xlarge-v2-squad-v2")

/Users/zacharyjgs/Projects/sous-chef/api/venv/lib/python3.7/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [221]:
model = tr.AutoModelForQuestionAnswering.from_pretrained('ktrapeznikov/albert-xlarge-v2-squad-v2')

In [246]:
question = "What is racism"
text = """The BEST and EASIEST Coffee Cake Recipe! It's moist, delicious and perfect for showers, breakfast and holiday celebrations!
Course Breakfast
Cuisine American
Prep Time 5 minutes
Cook Time 25 minutes
Total Time 30 minutes
Servings 15
Calories 314 kcal
Author Lil' Luna
Ingredients
1 cup oil
2 eggs beaten
1 tsp vanilla
1 cup milk
1 cup sugar
3 cups flour
3 tsp baking powder
1/2 tsp salt
1 1/2 cup brown sugar
2 tsp cinnamon
1/2 cup margarine/butter melted
Instructions
In a large mixing bowl. combine oil, eggs, vanilla and milk together.

In a medium bowl, blend together sugar, flour, baking powder and salt.
Combine egg mixture with flour mixture. Pour half the batter into a lightly greased 9x13 pan.

In a medium bowl, prepare streusel by combining brown sugar and cinnamon.

Sprinkle half of streusel on top of the batter. Top with remaining batter and then sprinkle the remaining streusel on top.
Drizzle with melted margarine.
Bake, uncovered at 350 for 25-30 minutes."""

In [247]:
encoding = tokenizer.encode_plus(question, text)

In [248]:
encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [249]:
input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]

In [250]:
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

In [251]:
span_scores = start_scores.softmax(dim=1).log()[:,:,None] + end_scores.softmax(dim=1).log()[:,None,:]

In [252]:
ignore_score = span_scores[:,0,0]

In [253]:
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)

In [254]:
answer_tokens = all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores) + 1]

In [255]:
answer = ' '.join(map(lambda token: token.replace('▁', ''), answer_tokens))

In [256]:
answer

'[CLS]'

In [257]:
tokenizer.convert_tokens_to_string(answer_tokens)

'[CLS]'